In [1]:
import polars as pl
import altair

In [4]:
rides = pl.read_csv("data/20*.csv", columns=["start station id", "end station id", "start station name", "end station name", "start station longitude", "start station latitude", "end station longitude", "end station latitude"])
rides 

start station id,end station id,start station name,end station name,start station longitude,start station latitude,end station longitude,end station latitude
i64,i64,str,str,f64,f64,f64,f64
36,19,"""Boston Public …","""Buswell St. at…",-71.077303,42.349673,-71.105301,42.347241
36,19,"""Boston Public …","""Buswell St. at…",-71.077303,42.349673,-71.105301,42.347241
36,19,"""Boston Public …","""Buswell St. at…",-71.077303,42.349673,-71.105301,42.347241
36,19,"""Boston Public …","""Buswell St. at…",-71.077303,42.349673,-71.105301,42.347241
36,67,"""Boston Public …","""MIT at Mass Av…",-71.077303,42.349673,-71.093198,42.3581
110,88,"""Harvard Univer…","""Inman Square a…",-71.114025,42.376369,-71.101427,42.374035
107,176,"""Ames St at Mai…","""Lesley Univers…",-71.08822,42.3625,-71.119019,42.386748
141,90,"""Kendall Street…","""Lechmere Stati…",-71.082168,42.36356,-71.076529,42.370677
178,80,"""MIT Pacific St…","""MIT Stata Cent…",-71.101295,42.359573,-71.092053,42.361962


In [15]:
columns = ["id", "name", "longitude", "latitude"]
both_stations = []
for prefix in ["start", "end"]:
    prefix_columns = [f"{prefix} station {c}" for c in columns]
    rides_prefix = rides.select(prefix_columns).rename(dict(zip(prefix_columns, columns))).unique()
    both_stations.append(rides_prefix)

# There are a few test stations with longitude+latitude are 0.
stations = pl.concat(both_stations).unique().filter(pl.col("longitude") != 0.0)
print(f"{stations.height} rows, {len(stations.get_column('id').unique())} unique ids, {len(stations.get_column('name').unique())} unique names")

932 rows, 522 unique ids, 684 unique names


In [16]:
stations.write_csv("output/bluebike_stations_for_geolocating.csv")